In [37]:
import cv2
import numpy as np

img = cv2.imread("test_img.jpg")

cv2.imshow("window" , img)
cv2.waitKey(0)
img.shape

(361, 380, 3)

In [22]:
# function to show the whole list of pictures
def output(list):
    img_blue , img_green , img_red  = list[0].copy() , list[1].copy() , list[2].copy()
    cv2.putText(img_blue , "Blue channel" , (20 , 360) , 1, 3 , (0,255,255) , 2)
    cv2.putText(img_green , "Green channel" , (20 , 360) , 1, 3 , (0,0,0) , 2)
    cv2.putText(img_red , "Red channel" , (50 , 360) , 1, 3 , (0,0,0) , 2)
    stack = np.hstack((img_blue , img_green , img_red))
    return stack

In [23]:
# separte into 3 channels
colorList = np.array([img[:,:,0] , img[:,:,1] , img[:,:,2]])


# outputing the all 3 channel 

colored_stack = output(colorList)
cv2.imshow("window" , colored_stack)
cv2.waitKey(0)


-1

In [24]:
# applying 1st order derivative to find (Sobel)
itt = 0
edgeImgList = []
edgeDirList = [0 , 0 , 0]

while itt < 3:
    sobelX = cv2.Sobel(colorList[itt] , -1 , 1 , 0)
    sobelY = cv2.Sobel(colorList[itt] ,-1 , 0 , 1 )
    edgeImgList.append(cv2.addWeighted(sobelX , 0.5 , sobelY ,0.5 , 0 ))
    itt += 1

# output(list=edgeImgList)    

sobel_stack = np.hstack((edgeImgList[0] , edgeImgList[1] , edgeImgList[1]))
cv2.imshow("win" , sobel_stack)
cv2.waitKey(0)

templist = edgeImgList


In [28]:
# applying thresholding on given cannels
itt = 0

while itt < 3:
    for x in range(0 , colorList[0].shape[0]):
        for y in range(0 , colorList[1].shape[1]):
            if templist[itt][x][y] > 42:
                templist[itt][x][y] = 255
            else:
                templist[itt][x][y] = 0
    itt += 1

temp_stack = output(templist)
stack = np.vstack((sobel_stack,temp_stack))
cv2.imshow("win" , stack)
cv2.waitKey(0)


-1

In [29]:
# applying 1st order derivative to find (Canny)
itt = 0
cannyList = []

while itt < 3:
    cannyList.append(cv2.Canny(colorList[0], 50, 100))
    itt += 1

canny_stack = output(list=cannyList)    



In [30]:
# compare sobel and canny

stack = np.vstack((sobel_stack,canny_stack))
cv2.imshow("win" , stack)
cv2.waitKey(0)

-1

In [38]:
# Event lishner to find coordinates of seed point

seedPoint = []
def  click_event(event , x , y , flags , params):

    if event == cv2.EVENT_LBUTTONDOWN:

        print(x , " " , y)
        seedPoint.append(x)
        seedPoint.append(y)
    



In [39]:
# function to find 8-n neighbour of give pixel

def n_8(x, y , shape):
    n8List = []
    maxRangeX = shape[0]
    maxRangeY = shape[1]

    if x-1 > 0 and y-1>0:
        n8List.append((x-1 , y-1))
    if x-1> 0:
        n8List.append((x-1 , y))
    if y-1>0 :
        n8List.append((x, y-1))
    if x+1 < maxRangeX :
        n8List.append((x+1 , y))
    if y+1 < maxRangeY :
        n8List.append((x , y+1))
    if x+1 < maxRangeX and y+1 < maxRangeY:
        n8List.append((x+1 , y+1))
    if x-1 > 0 and y+1 < maxRangeY:
        n8List.append((x-1 , y+1))
    if x+1 < maxRangeX and y-1 >0:
        n8List.append((x+1 , y-1)) 

    return n8List


    

In [40]:
# function for region growing threshold = 10

def region_grow(img , seedList):
    seedPoint = []
    outputImg = np.zeros_like(img)
    seedPoint.append((seedList[0] , seedList[1]))
    alreadyVisited = []
    print(seedList[0])

    while(len(seedPoint) > 0):
        pixel = seedPoint[0]
        outputImg[pixel[0] , pixel[1]] = 255
        for corr in n_8(pixel[0] , pixel[1] , img.shape):
            if outputImg[corr[0] , corr[1]] != 0:
                outputImg[corr[0] , corr[1]] = 255
                if not corr in alreadyVisited:
                    seedPoint.append(corr)
                alreadyVisited.append(corr)
        seedPoint.pop(0) 
        # cv2.imshow("porgress" , outputImg)
        # cv2.waitKey(1)
    return outputImg   


        


In [43]:
def is_simliar(pix , neighbour , threshold):
    return abs(int(pix) - int(neighbour)) <= threshold

def region_growing(img , seeds , theshold):

    outImg = np.zeros_like(img)

    region  = set(seeds)

    while seeds:

        seed = seeds.pop()

        x, y = seed[0] , seed[1]

        neighbors = n_8(x , y , img.shape)

        for nx , ny in neighbors:

            if is_simliar(img[x , y] , img[nx , ny] , theshold) and (nx , ny) not in region:
                region.add((nx , ny))
                outImg[nx , ny] = 255
                seeds.append((nx , ny))
    
    return outImg


    


In [44]:
cv2.namedWindow("win")
cv2.setMouseCallback("win" , click_event)
cv2.imshow("win" , cannyList[0])
cv2.waitKey(0)
# print(seedPoint)
abc = n_8(seedPoint[0] ,seedPoint[1], stack.shape)
print(abc)

out = region_growing(colorList[0] , seedPoint , 10)
cv2.imshow("window" , out)
cv2.waitKey(0)



189   160
189   160
[(174, 188), (174, 189), (175, 188), (176, 189), (175, 190), (176, 190), (174, 190), (176, 188)]


TypeError: 'int' object is not subscriptable

In [19]:
# stack = np.vstack((sobel_stack,canny_stack))
# cv2.namedWindow("win")
# cv2.setMouseCallback("win" , click_event)
# cv2.imshow("win" , stack)
# cv2.waitKey(0)
# print(seedPoint)
# print(n_8(seedPoint[0] ,seedPoint[1], stack.shape))


In [34]:
cv2.namedWindow("win")
cv2.setMouseCallback("win" , click_event)
cv2.imshow("win" , cannyList[0])
cv2.waitKey(0)
# print(seedPoint)
abc = n_8(seedPoint[0] ,seedPoint[1], stack.shape)
print(abc)



15   11
[(14, 10), (14, 11), (15, 10), (16, 11), (15, 12), (16, 12), (14, 12), (16, 10)]


In [35]:
out = region_grow(colorList[0] , seedPoint)
cv2.imshow("window" , out)
cv2.waitKey(0)

15


-1